In [49]:
import random
from base64 import b64decode
from json import loads
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
import sklearn
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()

In [50]:
def ler(x):

    digit = loads(x)
    array = np.fromstring(b64decode(digit["data"]),dtype=np.ubyte)
    array = array.astype(np.float64)
    return (digit["label"], array)

In [51]:
with open("digits.base64.json","r") as f:
    digits = map(ler, f.readlines())
digits=list(digits)

In [52]:
def display_digit(digit, labeled = True, title = ""):
    if labeled:
        digit = digit[1]
    image = digit
    plt.figure()
    fig = plt.imshow(image.reshape(28,28))
    fig.set_cmap('gray_r')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    if title != "":
        plt.title("Inferred label: " + str(title))

In [53]:
new_digits=[]
cont=0
while cont<len(digits):
    new_digits.append(digits[cont][1])
    cont+=1

In [54]:
digits_treino=digits[:len(digits)*5//6]
digits_val=digits[len(digits)*5//6:]
new_digits_treino=new_digits[:len(new_digits)*5//6]
new_digits_val=new_digits[len(new_digits)*5//6:]

In [55]:
r=[]
y=[]
cont=0
while cont<len(digits):
    r.append(digits[cont][0])
    cont+=1
y=[[[0] for i in np.arange(0,10)] for j in np.arange(0,60000)]
cont=0
while cont<len(digits):
    y[cont][r[cont]][0]=1
    cont+=1

Neural Network

In [56]:
n1=784
n2=30
n3=10
size=[n1,n2,n3]

In [57]:
def apply_sig(z,deriv=False):
    cont=0
    while cont<len(z):
        z[cont]=sig(z[cont],deriv)
        cont+=1
    return z

In [58]:
def plus(dp,b):
    cont=0
    z=[]
    while cont<len(b):
        z.append(dp[cont]+b[cont][0])
        cont+=1
    return z

In [59]:
def minus(a,y):
    cont=0
    r=[]
    while cont<len(a):
        r.append(a[cont][0]-y[cont][0])
        cont+=1
    return r

In [60]:
def sig(x, deriv=False):
    
    if deriv:
        return sig(x)*(1-sig(x))
    
    return 1/(1+np.exp(-x))

In [61]:
def feedforward(i,w,b):
    z = []
    a = []
    a.append(i)
    cont=0
    while cont<2:
        #print(0)
        #print(w[cont])
        #print(1)
        #print(a[cont])
        #print(2)
        dp = np.dot(w[cont],a[cont])
        #print(dp)
        #print(3)
        #print(b[cont])
        #print(4)
        #print(plus(dp,b[cont]))
        #print(5)
        z.append(plus(dp,b[cont]))
        #print(apply_sig(z[cont]))
        var = apply_sig(z[cont])
        a.append(var)
        cont+=1
    return a,z

In [95]:
w=[]
b=[]
w0 = np.random.randn(30,784)
w1 = np.random.randn(10,30)
w.append(w0)
w.append(w1)
b0 = np.random.randn(30,1)
b1 = np.random.randn(10,1)
b.append(b0)
b.append(b1)
i=new_digits[0]
a,z=feedforward(i,w,b)
a[2]

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


[5.383621861620514e-05,
 0.99986649933689875,
 0.9985069290361398,
 0.9984349981963444,
 0.99352514552749693,
 0.033538960627020137,
 0.044142430000642323,
 0.91415271886063088,
 0.28471170125742606,
 0.98795346324692523]

In [63]:
# i=new_digits[0]
# z = []
# a = []
# a.append(i)
# cont=0
# while cont<2:
#     dp = np.dot(w[cont],a[cont])
#     #print(w[cont])
#     #print(0)
#     print(a[cont])
#     print(b[cont])
#     print(1)
#     print(dp)
#     print(2)
#     z.append(plus(dp,b[cont]))
#     print(z[cont])
#     print(len(z[cont]))
#     #print(z[cont])
#     #print(3)
#     var = apply_sig(z[cont])
#     print(var)
#     a.append(var)
#     cont+=1
# a[2]

In [64]:
# w=[]
# b=[]
# w0 = np.random.randn(30,784)
# w1 = np.random.randn(10,30)
# w.append(w0)
# w.append(w1)
# b0 = np.random.randn(30,)
# b1 = np.random.randn(10,)
# b.append(b0)
# b.append(b1)
# i = new_digits[0]
# ava=feedforward(i,w,b)

In [65]:
def custox(a, cont):
    var = a[2]-y[cont]
    var = np.linalg.norm(var)
    cx = (var**2)/2
    return cx

In [66]:
def BP(a,z,cont):
    delta=[[],[]]
    #BP1
    #print(6)
    #print(a[2])
    #print(a[2][1][0])
    #print(7)
    #print(y[cont])
    #print(y[cont][1][0])
    #print(11)
    var=minus(a[2],y[cont])
    var2=apply_sig(z[1],deriv=True)
    delta[1]=np.multiply(var,var2)
    #BP2
    var=np.dot(w[1].transpose(),delta[1])
    var2=apply_sig(z[0],deriv=True)
    delta[0]=np.multiply(var,var2)
    #BP3
    deriv_b=[[],[]]
    cont=0
    while cont<len(delta[0]):
        deriv_b[0].append(delta[0][cont])
        cont+=1
    cont=0
    while cont<len(delta[1]):
        deriv_b[1].append(delta[1][cont])
        cont+=1
    #BP4
    deriv_w = [0,0]
    matrix = [[0 for i in np.arange(0,n1)] for j in np.arange(0,n2)]
    cont=0
    while cont<len(delta[0]):
        cont2=0
        while cont2<len(a[0]):
            matrix[cont][cont2] = a[0][cont2][0]*delta[0][cont][0]
            #if cont == 0 and cont2 == 0:
                #print(a[0][cont2])
                #print(delta[0][cont])
            cont2+=1
        cont+=1
    deriv_w[0]=matrix
    
    matrix = [[0 for i in np.arange(0,n2)] for j in np.arange(0,n3)]
    cont=0
    while cont<len(delta[1]):
        cont2=0
        while cont2<len(a[1]):
            matrix[cont][cont2] = a[1][cont2][0]*delta[1][cont][0]
            cont2+=1
        cont+=1
    deriv_w[1]=matrix
    return deriv_w, deriv_b

In [67]:
def mini_batch(l,w,b):
    l_deriv_w0=[]
    l_deriv_w1=[]
    l_deriv_b0=[]
    l_deriv_b1=[]
    cont=0
    while cont<len(l):
        i=np.resize(new_digits[l[cont]],(784,1))
        a,z = feedforward(i,w,b)
        deriv_w, deriv_b = BP(a,z,l[cont])
        l_deriv_w0.append(deriv_w[0])
        l_deriv_w1.append(deriv_w[1])
        l_deriv_b0.append(deriv_b[0])
        l_deriv_b1.append(deriv_b[1])
        cont+=1
    return l_deriv_w0,l_deriv_w1, l_deriv_b0, l_deriv_b1
        
    

In [68]:
def epoch(eta,w,b):
    passo=0
    todos=np.arange(0,50000)
    todos=np.random.shuffle(todos)
    while len(todos) != 0:
        l_deriv_w0,l_deriv_w1, l_deriv_b0, l_deriv_b1 = mini_batch(todos[:100],w,b)
        todos=todos[100:]
        mean_deriv_w0=l_deriv_w0[0]
        mean_deriv_w1=l_deriv_w1[0]
        mean_deriv_b0=l_deriv_b0[0]
        mean_deriv_b1=l_deriv_b1[0]
        cont=1
        while cont < len(l_deriv_w0):
            mean_deriv_w0 = np.add(mean_deriv_w0,l_deriv_w0[cont])
            cont+=1
        mean_deriv_w0 = np.multiply(eta/len(l_deriv_w0),mean_deriv_w0)
        
        cont=1
        while cont < len(l_deriv_w1):
            mean_deriv_w1 = np.add(mean_deriv_w1,l_deriv_w1[cont])
            cont+=1
        mean_deriv_w1 = np.multiply(eta/len(l_deriv_w1),mean_deriv_w1)
        
        cont=1
        while cont < len(l_deriv_b0):
            mean_deriv_b0 = np.add(mean_deriv_b0,l_deriv_b0[cont])
            cont+=1
        mean_deriv_b0 = np.multiply(eta/len(l_deriv_b0),mean_deriv_b0)
        
        cont=1
        while cont < len(l_deriv_b1):
            mean_deriv_b1 = np.add(mean_deriv_b1,l_deriv_b1[cont])
            cont+=1
        mean_deriv_b1 = np.multiply(eta/len(l_deriv_b1),mean_deriv_b1)
        print(w[0])
        print(1)
        print(mean_deriv_w0)
        w[0] = np.subtract(w[0],mean_deriv_w0)
        print(2)
        print(w[0])
        w[1] = np.subtract(w[1],mean_deriv_w1)
        print(3)
        print(b[0])
        print(4)
        print(mean_deriv_b0)
        b[0] = np.subtract(b[0],mean_deriv_b0)
        print(5)
        print(b[0])
        print(6)
        b[1] = np.subtract(b[1],mean_deriv_b1)
        passo+=100
        if passo%5000==0:
            print(passo)
    return w,b

In [92]:
w=[]
b=[]
w0 = np.random.randn(30,784)
w1 = np.random.randn(10,30)
w.append(w0)
w.append(w1)
b0 = np.random.randn(30,1)
b1 = np.random.randn(10,1)
b.append(b0)
b.append(b1)
w,b=epoch(3,w,b)

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


KeyboardInterrupt: 

In [76]:
cont=0
acertos=0
erros=0
while cont<50000:
    i=new_digits[cont]
    a,z=feedforward(i,w,b)
    m=max(a[2])
    cont2=0
    while cont2<len(a[2]):
        if m==a[2][cont2]:
            break
        cont2+=1
    if r[cont]==cont2:
        acertos+=1
    else:
        erros+=1
    cont+=1
(acertos,erros)

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


(5646, 44354)

In [91]:
b=[]
b0 = np.random.randn(30,1)
b1 = np.random.randn(10,1)
b.append(b0)
b.append(b1)

array([[ -1.49068823e+00,  -3.49716644e-01,  -1.01852355e+00,
          8.51607796e-01,  -8.64003013e-01,  -2.59791681e-01,
         -1.21081351e+00,   1.55416503e+00,  -1.21614517e+00,
          4.29074993e-01,  -9.99404885e-01,  -1.07797237e+00,
          2.87585959e-01,  -2.00008863e+00,   4.77524808e-01,
          4.67001270e-01,   5.87840984e-03,  -5.03751778e-01,
         -7.42902309e-01,   6.12971145e-01,  -4.32032331e-01,
         -6.72845818e-01,  -5.46385695e-01,   5.05289871e-01,
         -1.51071185e+00,  -1.38966188e+00,   7.25476941e-01,
         -2.03072407e+00,   4.41959670e-01,  -1.75859228e-01],
       [ -2.35602062e+00,  -2.11749117e+00,   3.83804729e-01,
         -9.88441800e-01,  -2.88542246e+00,   4.71694203e-01,
         -1.20021636e+00,   3.16709037e-03,  -1.07787485e+00,
         -1.10657134e+00,   1.44871693e+00,  -8.18840702e-01,
          8.62799020e-02,  -1.56469370e-01,  -7.78570945e-01,
          1.69481447e-01,  -2.22558525e+00,  -1.47189877e+00,
       

In [70]:
i=new_digits[0]
a,z=feedforward(i,w,b)

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


In [71]:
a[2]

[0.27598397132726393,
 0.00010052320436998881,
 0.99848171001478747,
 0.048734035566415645,
 0.24140383732089463,
 0.0062216771415414401,
 0.0013226000553016328,
 0.00011955108287633742,
 0.99786745797891574,
 0.00029511052887722176]

In [320]:
i=new_digits[0]
a,z=feedforward(i,w,b)

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


In [190]:
i=new_digits[0]
a,z=feedforward(i,w,b)
len(a[2][0])

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


30

In [73]:
i=new_digits[0]
a,z=feedforward(i,w,b)
r[0]

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


5

In [74]:
a

[array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
      

In [104]:
a[2]

array([[  3.28182316e-01,   4.11751792e-01,   3.28182316e-01,
          3.28182316e-01,   3.28182316e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          3.28182316e-01,   3.28182316e-01,   3.28182316e-01,
          3.28182316e-01,   4.11751792e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          3.28182316e-01,   4.11751792e-01,   4.11751792e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          4.11751792e-01,   4.11751792e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   3.28182316e-01],
       [  3.43415610e-01,   1.55810949e-04,   3.43415610e-01,
          3.43415610e-01,   3.43415610e-01,   3.43415610e-01,
          1.55810949e-04,   3.43415610e-01,   1.55810949e-04,
          3.43415610e-01,   3.43415610e-01,   3.43415610e-01,
          3.43415610e-01,   1.55810949e-04,   3.43415610e-01,
          1.55810949e-04,   3.43415610e-01,   1.55810949e-04,
       

In [42]:
[1,2,3,4,5,6,7,8,9][:4]
a=[1,2,3,4,5,6,7,8,9][4:]
a[4:]
eta=3

[9]

In [74]:
aa=[[1 for i in np.arange(0,10)] for i in np.arange(0,10)]
np.matrix(aa)
ab=[[1 for i in np.arange(0,10)] for i in np.arange(0,10)]
np.matrix(ab)
#np.add(aa,ab)
np.subtract(aa,ab)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [348]:
ff=np.array([[1,2,3],[4,5,6]])
np.resize(ff,(1,6))


array([[1, 2, 3, 4, 5, 6]])

In [300]:
s=np.matrix('1,2;1,2')
ss=np.matrix('1,2;1,2')
np.dot(s,ss)

matrix([[3, 6],
        [3, 6]])

In [302]:
kk=[[ 3495.93305194]
 [ 2309.44205931]
 [-1538.43166525]
 [-4354.97176587]
 [-1722.1941443 ]
 [ -116.45783507]
 [  511.60215533]
 [-1865.86281056]
 [-3405.56526365]
 [-2026.9890001 ]
 [ -653.545974  ]
 [ 1262.77805592]
 [-3273.07703311]
 [ 1800.15452869]
 [-1289.69855766]
 [-2401.52034368]
 [-2761.8326662 ]
 [   61.10424664]
 [ -343.83145781]
 [-1160.72317054]
 [-2545.96158699]
 [ 2951.45933329]
 [ 1627.13837676]
 [ 1204.17734677]
 [ 4157.50499083]
 [ 2939.26044339]
 [  981.93032985]
 [-3940.7161395 ]
 [-6506.05743237]
 [-4382.93828813]]
kkk=[-0.83531214
     -1.03067013 
     0.29784984
     0.62321164
     -1.7832455 
     -0.44610049
  0.23478798  0.82816775  0.10016847  0.37661515 -0.02132568  0.53276125
 -1.22000654  0.79959899 -0.72478409  0.05631258 -0.75506897  0.15592068
  0.01298866 -0.46866541 -0.15002919  1.40475922  1.30763051  0.32020513
 -0.11282627  0.98082362  1.87972589  0.80133476  0.05545351  1.10746091]
np.add(kk,kkk)


SyntaxError: invalid syntax (<ipython-input-302-e073c3f4165e>, line 33)